<h1>Atom Evaluation and Execution</h1>

In OpenCog, there are evaluatable and executable Atoms.

Some of them are <b>black-box</b> links (i.e. it's not clear what exactly it will just by reading the names of the Atoms) and some are <b>clear-box</b> links, e.g. `GreaterThanLink`, `EqualLink`, `AbsentLink` and `DeleteLink` etc.

We will explore both of them in this notebook.

<b>Note:</b> It's recommended to have some basic knowledge of atomese operations and pattern matching before running this notebook. See <b>atomese.ipynb</b> and <b>pattern_matching.ipynb</b> for details.

In [ ]:
import common
from opencog.utilities import initialize_opencog
from opencog.atomspace import AtomSpace, types
from opencog.type_constructors import *
from opencog.bindlink import execute_atom, evaluate_atom

atomspace = AtomSpace()
initialize_opencog(atomspace)

An example of a <b>clear-box</b> link:

In [ ]:
plus_link = PlusLink(NumberNode("1"), NumberNode("2"))
print(plus_link)

Use `execute_atom` to get the result:

In [ ]:
result = execute_atom(atomspace, plus_link)
print(result)

They can also be nested together:

In [ ]:
times_link = TimesLink(NumberNode("3"), NumberNode("0.5"))
greater_than_link = GreaterThanLink(plus_link, times_link)
print(greater_than_link)

This time, in order to get the result, we will have to use `evaluate_atom` instead since we are checking if the first number is greater than the second one:

In [ ]:
result = evaluate_atom(atomspace, greater_than_link)
print(result)

A <b>black-box</b> link is typically the one that will execute custom codes written in either <b>C++</b>, <b>Scheme</b>, or <b>Python</b>.

Since it's hard to tell what exactly the code will do before actually running it, it's basically a black-box for that reason.

Let's try to create one by firstly defining our own function:

In [ ]:
def check_tv(atom):
  val = atom.tv.mean * atom.tv.confidence
  if val > 0.5:
    return TruthValue(1, 1)
  else:
    return TruthValue(0, 1)

<b>Note:</b> It's expected to return a Truth Value for a evaluatable clause.

Now populate the AtomSpace with some knowledge:

In [ ]:
inh_1 = InheritanceLink(ConceptNode("Bob"), ConceptNode("cat"))
inh_2 = InheritanceLink(ConceptNode("Roy"), ConceptNode("dog"))

inh_1.tv = TruthValue(0.8, 0.5)
inh_2.tv = TruthValue(0.9, 0.7)

print("{}{}".format(inh_1, inh_2))

To turn it into an evaluatable link, we need to use an `EvaluationLink` and a `GroundedPredicateNode`.

The name of the `GroundedPredicateNode` that calls a custom function has to follow the following syntax:

<b>[written-language]: [function-name]</b>

In [ ]:
gpn = GroundedPredicateNode("py: check_tv")
print(gpn)

Give it a quick try and check the result:

In [ ]:
result = evaluate_atom(atomspace, EvaluationLink(gpn, ListLink(inh_1)))
print(result)

Similarity, to create an executable link, we can start with the custom function:

In [ ]:
def change_tv(atom):
  atom.tv = TruthValue(0.99, 0.99)
  return atom

<b>Note:</b> It's expected to return an Atom for a executable clause.

This time, we need to use an `ExecutionOutputLink` and a `GroundedSchemaNode` for creating an executable link.

The `GroundedSchemaNode` should follow the same <b>[written-language]: [function-name]</b> syntax.

In [ ]:
gsn = GroundedSchemaNode("py: change_tv")
print(gsn)

Execute it to see the result:

In [ ]:
result = execute_atom(atomspace, ExecutionOutputLink(gsn, ListLink(inh_2)))
print(result)